#### A job contains multiple tasks, different tasks executes different computing logics. Tasks form a DAG according to the data dependency. Instance is the smallest scheduling unit of batch workload. All instances within a task execute exactly the same binary with the same resource request, but with different input data.

In [39]:
import numpy as np
import pandas as pd 

In [3]:
bt = pd.read_csv("D:/Files/2019 Spring/High P Com/Project/trace_201708/trace_201708/batch_task.csv") 
bi = pd.read_csv("D:/Files/2019 Spring/High P Com/Project/trace_201708/trace_201708/batch_instance.csv")

In [4]:
bt.head()

,create_timestamp,modify_timestamp,job_id,task_id,instance_num,status,plan_cpu,plan_mem
0,6459,6524,3,4,15740,Terminated,50.0,0.007957
1,6457,6533,3,5,1,Terminated,50.0,0.004395
2,6036,6046,4,7,393,Waiting,NaN,NaN
3,6036,6046,4,6,452,Waiting,NaN,NaN
4,10719,11332,15,67,1705,Terminated,50.0,0.005736


In [5]:
bi.head()

,start_timestamp,end_timestamp,job_id,task_id,machineID,status,seq_no,total_seq_no,real_cpu_max,real_cpu_avg,...,120,686,299,Terminated,1,1.1,1.5,0.29,Unnamed: 20,Unnamed: 21
0,41561,41619.0,120.0,686.0,1279.0,Terminated,1.0,1.0,0.89,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41562,41617.0,120.0,686.0,828.0,Terminated,1.0,1.0,0.94,0.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41561,41617.0,120.0,686.0,95.0,Terminated,1.0,1.0,1.00,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41557,41610.0,120.0,686.0,545.0,Terminated,1.0,1.0,1.37,0.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41557,41614.0,120.0,686.0,258.0,Terminated,1.0,1.0,1.18,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
bt_job_id = np.sort(bt.job_id)
bt_job_id

array([    1,     2,     2, ..., 12951, 12951, 12951], dtype=int64)

In [20]:
bt_job_id[:20]

array([1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
      dtype=int64)

In [55]:
bt_task_pre_job = {}
bt_task_pre_job = pd.value_counts(bt_job_id)

In [56]:
max(bt_task_pre_job)

156

In [57]:
min(bt_task_pre_job)

1

In [58]:
sum(bt_task_pre_job) / len(bt_task_pre_job)

6.219828584665277

In [67]:
first_task_pre_job_createtime = {}
for i in range(1,12951):
    first_task_pre_job_createtime[i] = min(bt[bt.job_id == i].create_timestamp)

In [68]:
task_starttime = np.sort(bt.create_timestamp)
task_starttime

array([-1078, -1078, -1078, ..., 59763, 59798, 60181], dtype=int64)

In [70]:
last_task_pre_job_endtime = {}
for i in range(1,12951):
    last_task_pre_job_endtime[i] = max(bi[bi.job_id == i].end_timestamp)
last_task_pre_job_endtime

{1: 550.0,
 2: 2954.0,
 3: 6533.0,
 4: 0.0,
 5: 6386.0,
 6: 6592.0,
 7: 12370.0,
 8: 8070.0,
 9: 7364.0,
 10: 8263.0,
 11: 9779.0,
 12: 8570.0,
 13: 14098.0,
 14: 14324.0,
 15: 12813.0,
 16: 11217.0,
 17: 10985.0,
 18: 14247.0,
 19: 14507.0,
 20: 12946.0,
 21: 14333.0,
 22: 13124.0,
 23: 14862.0,
 24: 15177.0,
 25: 14632.0,
 26: 18034.0,
 27: 16558.0,
 28: 16391.0,
 29: 17026.0,
 30: 17147.0,
 31: 21466.0,
 32: 18514.0,
 33: 18621.0,
 34: 20967.0,
 35: 21735.0,
 36: 23155.0,
 37: 0.0,
 38: 21601.0,
 39: 23820.0,
 40: 22797.0,
 41: 23452.0,
 42: 23500.0,
 43: 22720.0,
 44: 29473.0,
 45: 27872.0,
 46: 26698.0,
 47: 23550.0,
 48: 21665.0,
 49: 22259.0,
 50: 21703.0,
 51: 23595.0,
 52: 28385.0,
 53: 24163.0,
 54: 26412.0,
 55: 24796.0,
 56: 24733.0,
 57: 24468.0,
 58: 24730.0,
 59: 24787.0,
 60: 25447.0,
 61: 24927.0,
 62: 25151.0,
 63: 26085.0,
 64: 29664.0,
 65: 26062.0,
 66: 26394.0,
 67: 26016.0,
 68: 27365.0,
 69: 28755.0,
 70: 27853.0,
 71: 28340.0,
 72: 30694.0,
 73: 30814.0,
 74: 4

In [73]:
runtime_pre_job = {}
for i in range(1,12951):
    runtime_pre_job[i] = last_task_pre_job_endtime[i] - first_task_pre_job_createtime[i]

In [74]:
max(runtime_pre_job)

12950

In [75]:
min(runtime_pre_job)

1

In [86]:
sum(runtime_pre_job) / len(runtime_pre_job)

6475.5

ValueError: If using all scalar values, you must pass an index